In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

import getpass
pd.options.mode.chained_assignment = None

In [4]:
# Ruta para pcs
user = getpass.getuser() # Capturo el usuario local / Ejemplo "Llan_", este es el nombre de mi usuario que inicie sesion en la pc

if   user== "USUARIO": # laptop @hugoferquiroz
     work = Path(r"D:\\Trabajo\Brecha-PRITE")
     data = Path(r"D:\\OneDrive\\Bases de datos\\Minedu compartido")
elif user== "hugo_": # laptop @hugoferquiroz
     work = Path(r"C:\\Users\\hugo_\\OneDrive - Ministerio de Educación\\MINEDU_2022\\RACIO")
     data = Path(r"C:\\OneDrive\\Bases de datos\\Minedu compartido")

# Construcción de la base de datos

## Matricula

In [5]:
# Importamos las bases de datos
df_mat_20 = pd.read_excel(data/r'matricula prite\\AC - Reporte de matricula por edad en los PRITE 2020-2022 - SIAGIE.xlsx',sheet_name='2020',skiprows=3)
df_mat_21 = pd.read_excel(data/r'matricula prite\\AC - Reporte de matricula por edad en los PRITE 2020-2022 - SIAGIE.xlsx',sheet_name='2021',skiprows=3)
df_mat_22 = pd.read_excel(data/r'matricula prite\\AC - Reporte de matricula por edad en los PRITE 2020-2022 - SIAGIE.xlsx',sheet_name='2022',skiprows=3)

In [6]:
def clean_matricula(df, año):
    # Cambiar los nombres
    df.rename(columns={0:'Menor a 12 meses',
                        1:'Entre 12 meses a 1 año',
                        2:'Entre 1 año a 2 años',
                        3:'Entre 2 años a 3 años',
                        4:'Entre 3 años a 4 años',
                        5:'Entre 4 años a 5 años',
                        'TOTAL':'Matricula reportada'}, inplace=True) 
    # Eliminar columnas innecesarias    
    if 'ERROR EN REGISTRO' in  df.columns.to_list():
        df.drop(columns=['ERROR EN REGISTRO'], inplace=True)
    # Verificar la longitud del codigo modular
    df['COD_MOD'] = df['COD_MOD'].apply(lambda x: x.zfill(7))
    # Eliminar la ultima fila = fila del total
    df.drop(df[df['COD_MOD'].str.find('TOTAL')!=-1].index,inplace=True)
    # Completar los missing de la matricula con 0
    df.fillna(0, inplace=True)
    # Agregar el año de la base
    df['Año'] = año
    return df

In [7]:
# Limpio las bases
df_mat_2020 = clean_matricula(df_mat_20,2020)
df_mat_2021 = clean_matricula(df_mat_21,2021)
df_mat_2022 = clean_matricula(df_mat_22,2022)

In [8]:
# Consolido las bases
matricula = pd.concat([df_mat_2020,df_mat_2021])
matricula = pd.concat([matricula,df_mat_2022])

In [9]:
# Doy formato a las columnas
    ## Genero las columnas en el orden que quiero
columnas = matricula.columns.to_list()
columnas_matricula = [columnas[0]] + [columnas[-3]] + columnas[1:5] + columnas[-2:] + [columnas[-4]]
columnas_matricula
    ## Cambio el orden del DataFrame
matricula = matricula[columnas_matricula]
    ## Imputo 0 a los missing\
matricula.fillna(0,inplace=True)

In [10]:
matricula.sample(5)

,COD_MOD,Año,Menor a 12 meses,Entre 12 meses a 1 año,Entre 1 año a 2 años,Entre 2 años a 3 años,Entre 3 años a 4 años,Entre 4 años a 5 años,Matricula reportada
46,1516020,2022,18.0,26.0,35.0,0.0,0.0,0.0,79
92,1768217,2020,2.0,3.0,1.0,0.0,0.0,0.0,6
10,1027796,2020,5.0,13.0,24.0,0.0,0.0,0.0,42
72,1722990,2022,2.0,2.0,3.0,0.0,0.0,0.0,7
37,1386481,2021,24.0,21.0,27.0,0.0,0.0,0.0,72


Evolución de la matricula

In [11]:
matricula_año = matricula.groupby('Año', as_index=False)['Matricula reportada'].agg('sum')

In [12]:
matricula_año.to_excel(work/r'tablas\\Evolución de matricula.xlsx')

## Personal

NEXUS al 4 de setiembre del 2022

In [13]:
# Importo la base de datos
nexus = pd.read_stata(data/r'Nexus\\2022\\nexus_35sira.dta')

c:\Users\USUARIO\anaconda3\lib\site-packages\pandas\io\stata.py:1514: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [14]:
# Selecciono los ssee identificados como PRITE
ssee_prite = matricula[['COD_MOD']]
ssee_prite.drop_duplicates(inplace=True) 
personal = pd.merge(left= ssee_prite, right=nexus, left_on='COD_MOD', right_on='codmod', how='left', validate='1:m',indicator=True)

In [15]:
# Funcion para identificar los duplicados por orden de prioridad
def plaza_unica(dataframe,codigo_modular='codmod'):
    print('--------------------------------------------------------------')
    print('Transformación de variables')
    dataframe['codmod'] = dataframe['codmod'].apply(str)
    dataframe.rename(columns={'codmod':'cod_mod'},inplace=True)
    dataframe['cod_mod'] = dataframe['cod_mod'].apply(lambda x: x.zfill(7))
    print('1. Verificar cantidad de digitos código modular')
    
    dataframe = dataframe[(dataframe.sitlab != "B") & (dataframe.sitlab != "P")]
    print('2. Eliminar plazas bloqueadas y previstas')
    
    dataframe['prior_tipo'] = 1
    dataframe.loc[dataframe['tiporegistro']== 'EVENTUAL', 'prior_tipo'] = 2
    dataframe.loc[dataframe['tiporegistro']== 'PROYECTO', 'prior_tipo'] = 3
    dataframe.loc[dataframe['tiporegistro']== 'CUADRO DE HORAS', 'prior_tipo'] = 4
    dataframe.loc[dataframe['tiporegistro'].str.find('REEMPLAZO')!=-1, 'prior_tipo'] = 5
    print('3. Priorizacion del tipo de plaza')

    dataframe['prior_sitlab'] = 1
    dataframe.loc[(dataframe['sitlab']== 'F') | (dataframe['sitlab']== 'D') 
                 | (dataframe['sitlab']== 'E') | (dataframe['sitlab']== 'T'), 'prior_sitlab'] = 2
    dataframe.loc[(dataframe['sitlab']== 'C') | (dataframe['sitlab']== 'V'), 'prior_sitlab'] = 3
    print('4. Priorizacion de la situación laboral')

    dataframe['soplaza'] = 0
    dataframe.loc[((dataframe['estplaza'].str.find("SG")==-1) 
                  & (dataframe['estplaza'].str.find("CG")==-1) 
                  & (dataframe['estplaza'].str.find("ABAND")==-1)), 'soplaza'] = 1
    print('5. Identificación plaza con goce, sin goce')
    
    dataframe['sestpla'] = 0
    dataframe.loc[dataframe['estplaza']== 'ACTIV', 'sestpla'] = 1
    print('6. Identificación estado de la plaza')
    
    dataframe.loc[dataframe['jornlab']=='', 'jornlab'] = 99
    dataframe = dataframe.sort_values(by='jornlab', ascending=False)
    dataframe['sjornlb'] = dataframe['jornlab'].rank(method='dense', ascending=False)
    print('7. Ordenamiento descendente de la jornada laboral')

    print('--------------------------------------------------------------')
    dataframe = dataframe.sort_values(['descreg', 'nombreooii', 'cod_mod', 'codplaza', 'prior_tipo', 
                          'prior_sitlab', 'soplaza', 'sestpla', 'sjornlb'])
    dataframe['tipo_fin'] = dataframe.groupby(['cod_mod', 'codplaza']).cumcount()
    dataframe['tipo_fin'] = dataframe['tipo_fin'] + 1
    print('Identificación plaza única')
    print(dataframe.groupby(['tipo_fin'])['tipo_fin'].count())
    print('Plaza original tienen la etiqueta igual a 1')
    return dataframe


In [16]:
# Identifico los duplicados
personal = plaza_unica(personal)

--------------------------------------------------------------
Transformación de variables
1. Verificar cantidad de digitos código modular
2. Eliminar plazas bloqueadas y previstas
3. Priorizacion del tipo de plaza
4. Priorizacion de la situación laboral
5. Identificación plaza con goce, sin goce
6. Identificación estado de la plaza
7. Ordenamiento descendente de la jornada laboral
--------------------------------------------------------------
Identificación plaza única
tipo_fin
1    519
2     28
3      1
4      1
Name: tipo_fin, dtype: int64
Plaza original tienen la etiqueta igual a 1


In [17]:
# Elimino los duplicados
personal = personal.loc[(personal['tipo_fin']==1) | (personal['_merge']=='left_only')]

Personal por regimen laboral

In [18]:
pd.crosstab(personal['descargo'],personal['desley'],margins=True)

desley,D.L. 1153,D.L. 276,DL 1057,LEY 29944,LEY 30328,LEY 30493,All
descargo,,,,,,,
AUXILIAR DE EDUCACION,0,0,0,0,0,8,8
COORDINACION PEDAGOGICO,0,0,0,1,0,0,1
COORDINADOR ACADEMICO,0,0,0,5,0,0,5
DIRECTOR I.E.,0,0,0,33,0,0,33
DOCENTE COORDINADOR,0,0,0,0,1,0,1
FISIOTERAPEUTA,0,1,0,0,0,0,1
OFICINISTA,0,2,0,0,0,0,2
PROFESIONAL DE TERAPIA FÍSICA PARA LOS PROGRAMAS DE INTERVENCIÓN TEMPRANA,0,0,123,0,0,0,123
PROFESOR,0,0,0,121,146,0,267


In [19]:
# Exporto la tabla
(pd.crosstab(personal['descargo'],personal['desley'],margins=True)).to_excel(work/r'tablas\\personal por regimen laboral.xlsx')

Personal por tipo de trabajador

In [20]:
pd.crosstab(personal['descargo'],personal['desctipotrab'],margins=True)

desctipotrab,ADMINISTRATIVO,CAS,DOCENTE,PROFESIONAL DE LA SALUD,All
descargo,,,,,
AUXILIAR DE EDUCACION,0,0,8,0,8
COORDINACION PEDAGOGICO,0,0,1,0,1
COORDINADOR ACADEMICO,0,0,5,0,5
DIRECTOR I.E.,0,0,33,0,33
DOCENTE COORDINADOR,0,0,1,0,1
FISIOTERAPEUTA,1,0,0,0,1
OFICINISTA,2,0,0,0,2
PROFESIONAL DE TERAPIA FÍSICA PARA LOS PROGRAMAS DE INTERVENCIÓN TEMPRANA,0,123,0,0,123
PROFESOR,0,0,267,0,267


In [21]:
(pd.crosstab(personal['descargo'],personal['desctipotrab'],margins=True)).to_excel(work/r'tablas\\personal por tipo de trabajador.xlsx')

In [22]:
# Homogenizo los perfiles
    ## Docentes
personal['Profesor'] = 0
personal.loc[(personal['descargo']=='PROFESOR') | 
             (personal['descargo']=='PROFESOR (FUNCIONES DE DIRECTOR)') | 
             (personal['descargo']=='PROFESOR COORDINADOR') |
             (personal['descargo']=='COORDINACION PEDAGOGICO') |
             (personal['descargo']=='AUXILIAR DE EDUCACION') | 
             (personal['descargo']=='DOCENTE COORDINADOR') |        
             (personal['descargo']=='COORDINADOR ACADEMICO') ,'Profesor'] =1 
    ## Director
personal['Director'] = 0
personal.loc[(personal['descargo']=='DIRECTOR I.E.') ,'Director'] =1 
    ## Tecnologos medicos
personal['Tecnologo medico'] = 0
personal.loc[(personal['descargo']=='PROFESIONAL DE TERAPIA FÍSICA PARA LOS PROGRAMAS DE INTERVENCIÓN TEMPRANA') | 
             (personal['descargo']=='TERAPISTA') | 
             (personal['descargo']=='FISIOTERAPEUTA'),'Tecnologo medico'] =1     
    ## Psicologos
personal['Psicologo'] = 0
personal.loc[(personal['descargo'].str.find('PSICOLOGO')!=-1) ,'Psicologo'] =1 
    ## Personal administrativo
personal['Administrativos'] = 0
personal.loc[(personal['descargo']=='TRABAJADOR DE SERVICIO') | 
             (personal['descargo']=='TRABAJADOR DE SERVICIO II') | 
             (personal['descargo']=='TRABAJADOR DE SERVICIO III') | 
             (personal['descargo']=='OFICINISTA') |                              
             (personal['descargo']=='TECNICO ADMINISTRATIVO'),'Administrativos'] =1

In [23]:
# Colapso el DataFrame a nivel de servicio educativo
trabajadores = personal.groupby('COD_MOD', as_index=False).agg('sum')
# Doy formato a la base
columnas = trabajadores.columns.tolist()
columnas_nexus = [columnas[0]] + columnas[-5:]
trabajadores = trabajadores[columnas_nexus]
trabajadores.sample(5)

,COD_MOD,Profesor,Director,Tecnologo medico,Psicologo,Administrativos
47,1519958,6,0,1,0,0
81,1731942,1,0,1,0,0
24,1143668,5,0,1,0,1
35,1366350,2,1,1,0,0
115,1793991,0,0,0,1,0


## Datos del padron web
Padron web descargado de la [pagina web](http://escale.minedu.gob.pe/uee/-/document_library_display/GMv7/view/958881) de escale con fecha 17/09/2022 y complemetado con información de las asignaciones temporales y unidades ejecutoras de la Unidad de Presupuesto

In [24]:
# Importo el padron de la Unidad de Presupuesto 
df_padron = pd.read_stata(data/r'Padron GG1.dta')
df_padron = df_padron.loc[df_padron['niv_mod']=='E0']

c:\Users\USUARIO\anaconda3\lib\site-packages\pandas\io\stata.py:1514: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


In [25]:
# Identifico a los servicios PRITE
padron = pd.merge(left= ssee_prite, right=df_padron, left_on='COD_MOD', right_on='cod_mod', how='left', validate='1:m',indicator=True)
# Filtro las columnas relevantes
datos_padron = ['COD_MOD','cen_edu','d_gestion','d_ges_dep','ubigeo','d_dpto','d_prov', 'd_dist','d_region','codooii', 'd_dreugel','nlat_ie','nlong_ie',
                'vraem_upp_2022','tipie_upp_2022','bilin_upp_2022','rural_upp_2022','fron_upp_2022','region','pliego','codue', 'unidadejecutora', 'nombentidad']
padron_web = padron[datos_padron]

In [26]:
padron_web.sample(5)

,COD_MOD,cen_edu,d_gestion,d_ges_dep,ubigeo,d_dpto,d_prov,d_dist,d_region,codooii,...,vraem_upp_2022,tipie_upp_2022,bilin_upp_2022,rural_upp_2022,fron_upp_2022,region,pliego,codue,unidadejecutora,nombentidad
102,1788678,PRITE PITUMARCA,P£blica de gesti¢n directa,Sector Educaci¢n,080605,CUSCO,CANCHIS,PITUMARCA,DRE CUSCO,080006,...,No Vraem,Completo,No bilingue,Urbano,No frontera,CUSCO,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,446302,302. EDUCACION CANCHIS,REGION CUSCO-EDUCACION CANCHIS
36,1386473,PRITE SAN JUAN DE DIOS,P£blica de gesti¢n directa,Sector Educaci¢n,080101,CUSCO,CUSCO,CUSCO,DRE CUSCO,080001,...,No Vraem,Completo,No bilingue,Urbano,No frontera,CUSCO,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,446312,312. UGEL CUSCO,GOB. REG. DPTO. CUSCO - UGEL CUSCO
46,1516020,PRITE ALEGRIA EN EL CORAZON,P£blica de gesti¢n directa,Sector Educaci¢n,150141,LIMA,LIMA,SURQUILLO,DRE LIMA METROPOLITANA,150108,...,No Vraem,Completo,No bilingue,Urbano,No frontera,LIMA METROPOLITANA,10. LIMA METROPOLITANA,107,007. USE 07 SAN BORJA,USE 07 SAN BORJA
93,1772870,PRITE SAN FRANCISCO DE ASIS,P£blica de gesti¢n directa,Sector Educaci¢n,120302,JUNIN,CHANCHAMAYO,PERENE,DRE JUNIN,120004,...,No Vraem,Completo,No bilingue,Urbano,No frontera,JUNIN,450. GOBIERNO REGIONAL DEL DEPARTAMENTO DE JUNIN,450303,303. EDUCACION CHANCHAMAYO,GOB. REG. DE JUNIN- EDUCACION CHANCHAMAYO
33,1329135,PRITE CHEPEN,P£blica de gesti¢n directa,Sector Educaci¢n,130401,LA LIBERTAD,CHEPEN,CHEPEN,DRE LA LIBERTAD,130005,...,No Vraem,Completo,No bilingue,Urbano,No frontera,LA LIBERTAD,451. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LA ...,451301,301. EDUCACION CHEPEN,REGION LA LIBERTAD-EDUCACION CHEPEN


# Calculo de la brecha

## Consolidación de la base

In [27]:
df = pd.merge(left=padron_web, right=df_mat_2022, on='COD_MOD')
df = pd.merge(left=df, right=trabajadores, on='COD_MOD')
df.sample(3)

,COD_MOD,cen_edu,d_gestion,d_ges_dep,ubigeo,d_dpto,d_prov,d_dist,d_region,codooii,...,Entre 1 año a 2 años,Entre 2 años a 3 años,Entre 3 años a 4 años,Matricula reportada,Año,Profesor,Director,Tecnologo medico,Psicologo,Administrativos
84,1748193,PRITE TACNA,P£blica de gesti¢n directa,Sector Educaci¢n,230110,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN LANCHIPA,DRE TACNA,230001,...,4,0.0,0.0,18,2022,3,0,1,0,0
115,1794387,PRITE MI LINDA CASITA,P£blica de gesti¢n directa,Sector Educaci¢n,210101,PUNO,PUNO,PUNO,DRE PUNO,210001,...,4,0.0,0.0,10,2022,1,0,0,0,0
15,1057702,PRITE,P£blica de gesti¢n directa,Sector Educaci¢n,150108,LIMA,LIMA,CHORRILLOS,DRE LIMA METROPOLITANA,150108,...,42,0.0,0.0,100,2022,7,1,2,0,1


## Calculo los ratios de asignación

De acuerdo al [RVM 188-2019](https://cdn.www.gob.pe/uploads/document/file/348345/RVM_N__188-2019-MINEDU.pdf) como mínimo se asigna 1 personal docente, psicólogo y técnologo médico por cada 12 alumnos menores de 3 años. 

Los niños y niñas de 3 años de edad con discapacidad severa o multidiscapacidad pueden permanecer en el PRITE hasta los 5 años de edad, solo si se encuentran en una zona rural y/o bilingue o si en su jurisdicción no se cuanta con un CEBE

In [28]:
df['Personal segun norma'] = df['Matricula reportada']/12
df['Personal segun norma'] = df['Personal segun norma'].astype(int)

In [29]:
# Calculo los requerimientos y excedentes
ls_personal = ['Profesor','Tecnologo medico','Psicologo']

for prof in ls_personal:
    df[f'{prof} requerimiento'] = np.where(df[f'{prof}']<=df['Personal segun norma'],df['Personal segun norma']-df[f'{prof}'],0)
    df[f'{prof} excedente'] = np.where(df[f'{prof}']>=df['Personal segun norma'],df[f'{prof}']-df['Personal segun norma'],0)


df['Director norma'] = 1
df['Director requerimiento'] = np.where(df['Director']<=df['Director norma'],df['Director norma']-df['Director'],0)
df['Director excedente'] = np.where(df['Director']>=df['Director norma'],df['Director']-df['Director norma'],0)

## Estructura de costos

In [37]:
def table_costo(data_frame):
    # Calculo el costo
    w_psico, w_profe,w_tec_medico,w_director = 2500*12+600, 12*86.67*30*(1+0.096*0.65)+600, 2000*12+600, 12*86.67*40*(1+0.096*0.65)+600+600 
    data_frame['Costo-Profesor'] = data_frame['Brecha-Profesor']*w_profe
    data_frame['Costo-Director'] = data_frame['Brecha-Director']*w_director
    data_frame['Costo-Tecnologo medico'] = data_frame['Brecha-Tecnologo medico']*w_tec_medico
    data_frame['Costo-Psicologo'] = data_frame['Brecha-Psicologo']*w_profe

    # Doy formato a la tabla
    data_frame.loc['Total'] = data_frame.sum()
    data_frame_resumen = data_frame.style.format({'Costo-Profesor':'S/{:20,.0f}','Costo-Director':'S/{:20,.0f}','Costo-Tecnologo medico':'S/{:20,.0f}','Costo-Psicologo':'S/{:20,.0f}',
                                                  'Brecha-Profesor': '{:20,.0f}','Brecha-Director': '{:20,.0f}','Brecha-Tecnologo medico': '{:20,.0f}','Brecha-Psicologo': '{:20,.0f}'})
                                                                            
    return data_frame_resumen


## Brecha sin optimizar

En este escenario no se usan las plazas excedentes para cubrir los requerimientos identificados por lo que la brecha es equivalente a los requerimientos identificados 

In [38]:
# Colapso la brecha por region
sr = df.groupby(['region'])['Profesor requerimiento','Director requerimiento','Tecnologo medico requerimiento','Psicologo requerimiento'].sum()
sr.rename(columns={'Profesor requerimiento': 'Brecha-Profesor',
                   'Director requerimiento': 'Brecha-Director',
                   'Tecnologo medico requerimiento': 'Brecha-Tecnologo medico',
                   'Psicologo requerimiento': 'Brecha-Psicologo'},inplace=True)                           

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_27164\1724990708.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sr = df.groupby(['region'])['Profesor requerimiento','Director requerimiento','Tecnologo medico requerimiento','Psicologo requerimiento'].sum()


In [39]:
sr_region = table_costo(sr)
sr_region

,Brecha-Profesor,Brecha-Director,Brecha-Tecnologo medico,Brecha-Psicologo,Costo-Profesor,Costo-Director,Costo-Tecnologo medico,Costo-Psicologo
region,,,,,,,,
AMAZONAS,1,2,1,2,"S/ 33,748","S/ 90,795","S/ 24,600","S/ 67,496"
ANCASH,1,1,13,16,"S/ 33,748","S/ 45,398","S/ 319,800","S/ 539,970"
APURIMAC,4,7,1,9,"S/ 134,993","S/ 317,783","S/ 24,600","S/ 303,733"
AREQUIPA,2,3,7,12,"S/ 67,496","S/ 136,193","S/ 172,200","S/ 404,978"
AYACUCHO,0,1,4,5,S/ 0,"S/ 45,398","S/ 98,400","S/ 168,741"
CAJAMARCA,1,2,2,8,"S/ 33,748","S/ 90,795","S/ 49,200","S/ 269,985"
CALLAO,1,1,10,13,"S/ 33,748","S/ 45,398","S/ 246,000","S/ 438,726"
CUSCO,10,13,14,35,"S/ 337,482","S/ 590,168","S/ 344,400","S/ 1,181,185"
HUANCAVELICA,1,3,2,5,"S/ 33,748","S/ 136,193","S/ 49,200","S/ 168,741"


In [ ]:
sr_region.to_excel(work/r'tablas\\brecha sin racio.xlsx'

## Brecha optimizada 

En este escenario se usan las plazas excedentes para cubrir los requerimientos identificados siempre y cuando estos excedentes sean reubicados dentro de la region

In [42]:
# Calculo la brecha/requerimientos
df_copy = df.copy()
ls_cargos = ['Profesor',	'Director',	'Tecnologo medico', 'Psicologo']
for prof in ls_cargos:
    df_copy[f'Brecha-{prof}'] = df_copy[f'{prof}']-df_copy['Personal segun norma']

In [43]:
# Colapso la brecha por region
cr = df_copy.groupby(['region'])['Brecha-Profesor','Brecha-Director','Brecha-Tecnologo medico','Brecha-Psicologo'].sum()

for prof in ls_cargos:
    cr.loc[cr[f'Brecha-{prof}']>=0 , [f'Brecha-{prof}']] = 0
    cr.loc[cr[f'Brecha-{prof}']<0,f'Brecha-{prof}']  = np.abs(cr[f'Brecha-{prof}'])


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_27164\2681851656.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cr = df_copy.groupby(['region'])['Brecha-Profesor','Brecha-Director','Brecha-Tecnologo medico','Brecha-Psicologo'].sum()


In [44]:
# Costo 
cr_region = table_costo(cr)
cr_region

,Brecha-Profesor,Brecha-Director,Brecha-Tecnologo medico,Brecha-Psicologo,Costo-Profesor,Costo-Director,Costo-Tecnologo medico,Costo-Psicologo
region,,,,,,,,
AMAZONAS,0,2,0,2,S/ 0,"S/ 90,795",S/ 0,"S/ 67,496"
ANCASH,0,14,13,16,S/ 0,"S/ 635,566","S/ 319,800","S/ 539,970"
APURIMAC,4,8,1,9,"S/ 134,993","S/ 363,180","S/ 24,600","S/ 303,733"
AREQUIPA,0,9,7,12,S/ 0,"S/ 408,578","S/ 172,200","S/ 404,978"
AYACUCHO,0,5,4,5,S/ 0,"S/ 226,988","S/ 98,400","S/ 168,741"
CAJAMARCA,0,6,2,8,S/ 0,"S/ 272,385","S/ 49,200","S/ 269,985"
CALLAO,1,12,10,13,"S/ 33,748","S/ 544,770","S/ 246,000","S/ 438,726"
CUSCO,1,30,12,35,"S/ 33,748","S/ 1,361,926","S/ 295,200","S/ 1,181,185"
HUANCAVELICA,0,5,2,5,S/ 0,"S/ 226,988","S/ 49,200","S/ 168,741"


In [ ]:
# Exporto los resultados
cr_region.to_excel(work/r'tablas\\brecha con racio.xlsx')

# Priorizacion de recursos

## Priorizacion para completar el equipo

## Priorizacion de personal directivo

Se prioriza cubrir los requerimientos de director en los servicios PRITE que tengan el equipo interdisciplinario completo

In [51]:
df.loc[(df['Director requerimiento']!=0) & (df['Profesor']!=0) & (df['Tecnologo medico']!=0) & (df['Psicologo']!=0), ['Director requerimiento','Profesor','Tecnologo medico','Psicologo']]

,Director requerimiento,Profesor,Tecnologo medico,Psicologo
48,1,2,1,1


In [48]:
df[['Director requerimiento','Profesor','Tecnologo medico','Psicologo']]

,Director requerimiento,Profesor,Tecnologo medico,Psicologo
0,1,3,1,0
1,0,4,2,0
2,1,8,1,0
3,1,3,1,0
4,1,4,1,0
...,...,...,...,...
111,1,1,0,0
112,1,1,0,0
113,1,0,0,1
114,1,0,0,1


## Priorizacion de personal docente